In [1]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
f = open('../instance_files/instance_iceice_rand_rall_major_depth', 'rb')
results = pickle.load(f)
agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds, major_ax_zs, depths= \
                results['agg_as'], results['agg_bs'], results['agg_cs'], results['phi2Ds'], results['cplxs'], \
                results['dds'], results['major_ax_zs'], results['depths']
f.close()

#also need to do for flat

In [ ]:
def calculate_error(data, numaspectratios, ch):
    data[np.isinf(data)] = min(data)
    data[np.isnan(data)] = min(data)
    data= data[(data<np.quantile(data, .99)) & (data>np.quantile(data, .01))]
            
    mean = np.mean(data)
    std = np.std(data)
    shape = (mean/std)**2
    scale = (std**2)/mean
    shapech = mean/(numaspectratios*ch)

    pos_error = mean + std
    neg_error = mean - std

    min_data = min(data)
    max_data = max(data)

    return(pos_error, neg_error, min_data, max_data, mean)   

def mode_of_hist(data):
    data[np.isinf(data)] = min(data)
    data[np.isnan(data)] = min(data)
    data= data[(data<np.quantile(data, .99)) & (data>np.quantile(data, .01))]
    bins = (np.max(data) - np.min(data))/0.01
    n, bins, patches = plt.hist(data, bins=int(bins), density=True,
                                color='navy',range=(min(data), max(data)))
    mode = bins[np.where(n == np.max(n))]

    return mode[0]

def fit_distribution(data, normed = True, facecolor='navy', alpha=1.0, axes=None, **kwargs):
    data[np.isinf(data)] = min(data)
    data[np.isnan(data)] = min(data)
    data= data[(data<np.quantile(data, .99)) & (data>np.quantile(data, .01))]
    # Get histogram of original data
    y, x = np.histogram(data, density=True)
    fit_alpha, fit_loc, fit_beta=st.gamma.fit(data)
    pdf = st.gamma.pdf(x, a=fit_alpha, loc=fit_loc, scale=fit_beta)
    indmax = np.argmax(pdf) #FIRST index where the highest prob occurs
    gammach_ch = x[indmax] #characteristic of the distribution
    
    if axes is not None:
        n, bins, patches = plt.hist(data, bins=70, density=True,
                                color='navy',**kwargs)

        ax = plt.plot(x, pdf, lw=5, color='darkorange')
        plt.ylim(0,max(n))
        plt.show()

    return gammach_ch

In [ ]:
#find characteristic of gamma distribution
numaspectratios=len(agg_as[:,0,0])
numrs=len(agg_bs[0,:,0])
agg_cs_ch = np.empty((numaspectratios, numrs), dtype=np.float64) #major
agg_as_ch = np.empty((numaspectratios, numrs), dtype=np.float64) #minor
agg_as_mean = np.empty((numaspectratios, numrs), dtype=np.float64)
dds_mode = np.empty((numaspectratios, numrs), dtype=np.float64)
major_ax_zs_ch = np.empty((numaspectratios, numrs), dtype=np.float64)
major_ax_zs_mean = np.empty((numaspectratios, numrs), dtype=np.float64)
depths_ch = np.empty((numaspectratios, numrs), dtype=np.float64)

fig = plt.figure(figsize=(5,7))
ax = plt.subplot(131)

for i in range(numaspectratios):
    for r in range(numrs):
        #print(i,r)
        for c, data in enumerate([agg_cs, agg_as, dds, major_ax_zs, depths]):
        
            if c == 0:
                agg_cs_ch[i,r] = fit_distribution(data[i,r,:])
            if c == 1:
                agg_as_ch[i,r] = fit_distribution(data[i,r,:])
                pos_error, neg_error, min_data, max_data, agg_as_mean[i,r] = calculate_error(data[i,r,:], numaspectratios, agg_as_ch[i,r])
            if c == 2:
                #dds_ch[i,r] = fit_distribution(data[i,r,:])
                dds_mode[i,r] = mode_of_hist(data[i,r])
            if c == 3:
                major_ax_zs_ch[i,r] = fit_distribution(data[i,r,:])
                pos_error, neg_error, min_data, max_data, major_ax_zs_mean[i,r] = calculate_error(data[i,r,:], numaspectratios, major_ax_zs_ch[i,r])
            if c == 4:
                depths_ch[i,r] = fit_distribution(data[i,r,:])
                

In [ ]:
#write to file for output as array:
with open("newformat_minorax_ellipsoid_flat_ch.dat","w") as file1:
    with open("newformat_majorax_ellipsoid_flat_ch.dat","w") as file2:
        with open("newformat_majorax_ellipsoid_flat_mean.dat","w") as file3:
            with open("newformat_dd_flat_mode.dat","w") as file4:
                with open ("newformat_majorax_ellipse_z_flat_ch.dat","w") as file5:
                    with open ("newformat_majorax_ellipse_z_flat_mean.dat","w") as file6:
                        with open ("newformat_depth_flat_ch.dat","w") as file7:
                        
                            file1.write('Ice-Ice collection for the flat orientation. \n'\
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Minor axis taken as the smallest axis from the fit-ellipsoid \n'\
                                        'Order: phi, r, value \n')
                            
                            file2.write('Ice-Ice collection for the flat orientation. \n'
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Major axis taken as the largest axis from the fit-ellipsoid \n'
                                        'Order: phi, r, value \n')
                                
                            file3.write('Ice-Ice collection for the flat orientation. \n'
                                        'Mean value taken from the average across 300 aggregates. \n'\
                                        'Major axis taken as the largest axis from the fit-ellipsoid \n'
                                        'Order: phi, r, value \n')
                                
                            file4.write('Ice-Ice collection for the flat orientation. \n'
                                        'Volume ratio of monomer (not 1.0) subtracted from volume ratio of agg (Vagg/Vellipse)\n'\
                                        'then normalized by the volume ratio of monomer. \n'\
                                        'Mode from 300 aggregates. \n'\
                                        'Order: phi, r, value \n')
                                
                            file5.write('Ice-Ice collection for the flat orientation. \n'\
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Major axis taken from a fit-ellipse (2D) in the z-orientation. \n'
                                        'Order: phi, r, value \n')
                                
                            file6.write('Ice-Ice collection for the flat orientation. \n'
                                        'Mean value taken from the average across 300 aggregates. \n'\
                                        'Major axis taken from a fit-ellipse (2D) in the z-orientation. \n'
                                        'Order: phi, r, value \n')
                                
                            file7.write('Ice-Ice collection for the flat orientation. \n'
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Depth measured as max z point - min z point. \n'
                                        'Order: phi, r, value \n')

                            for i in range(len(phioarr)):
                                for r in range(len(reqarr)):
                                    #print(i,r)
                                    file1.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  agg_cs_ch[i,r]))
                                    file2.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  agg_as_ch[i,r]))
                                    file3.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  agg_as_mean[i,r]))
                                    file4.write('%.3f %.2f %.4f \n' %(phioarr[i], reqarr[r],  dds_mode[i,r]))
                                    file5.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  major_ax_zs_ch[i,r]))
                                    file6.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  major_ax_zs_mean[i,r]))
                                    file7.write('%.3f %.2f %.4f \n' %(phioarr[i], reqarr[r],  depths_ch[i,r]))

file1.close()
file2.close() 
file3.close()
file4.close()
file5.close()
file6.close()
file7.close()

In [ ]:
#write to file for output as array:
with open("newformat_minorax_ellipsoid_rand_ch.dat","w") as file1:
    with open("newformat_majorax_ellipsoid_rand_ch.dat","w") as file2:
        with open("newformat_majorax_ellipsoid_rand_mean.dat","w") as file3:
            with open("newformat_dd_rand_mode.dat","w") as file4:
                with open ("newformat_majorax_ellipse_z_rand_ch.dat","w") as file5:
                    with open ("newformat_majorax_ellipse_z_rand_mean.dat","w") as file6:
                        with open ("newformat_depth_rand_ch.dat","w") as file7:
                        
                            file1.write('Ice-Ice collection for the random orientation. \n'\
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Minor axis taken as the smallest axis from the fit-ellipsoid \n'\
                                        'Order: phi, r, value \n')
                            
                            file2.write('Ice-Ice collection for the random orientation. \n'
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Major axis taken as the largest axis from the fit-ellipsoid \n'
                                        'Order: phi, r, value \n')
                                
                            file3.write('Ice-Ice collection for the random orientation. \n'
                                        'Mean value taken from the average across 300 aggregates. \n'\
                                        'Major axis taken as the largest axis from the fit-ellipsoid \n'
                                        'Order: phi, r, value \n')
                                
                            file4.write('Ice-Ice collection for the random orientation. \n'
                                        'Volume ratio of monomer (not 1.0) subtracted from volume ratio of agg (Vagg/Vellipse)..\n'\
                                        'then normalized by the volume ratio of monomer. \n'\
                                        'Mode from 300 aggregates. \n'\
                                        'Order: phi, r, value \n')
                                
                            file5.write('Ice-Ice collection for the random orientation. \n'\
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Major axis taken from a fit-ellipse (2D) in the z-orientation. \n'
                                        'Order: phi, r, value \n')
                                
                            file6.write('Ice-Ice collection for the random orientation. \n'
                                        'Mean value taken from the average across 300 aggregates. \n'\
                                        'Major axis taken from a fit-ellipse (2D) in the z-orientation. \n'
                                        'Order: phi, r, value \n')
                                
                            file7.write('Ice-Ice collection for the random orientation. \n'
                                        'Characteristic values taken from the peak of a fit \n'\
                                        'gamma distribution from 300 aggregates. \n'\
                                        'Depth measured as max z point - min z point. \n'
                                        'Order: phi, r, value \n')

                            for i in range(len(phioarr)):
                                for r in range(len(reqarr)):
                                    #print(i,r)
                                    file1.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  agg_cs_ch[i,r]))
                                    file2.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  agg_as_ch[i,r]))
                                    file3.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  agg_as_mean[i,r]))
                                    file4.write('%.3f %.2f %.4f \n' %(phioarr[i], reqarr[r],  dds_mode[i,r]))
                                    file5.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  major_ax_zs_ch[i,r]))
                                    file6.write('%.3f %.2f %.2f \n' %(phioarr[i], reqarr[r],  major_ax_zs_mean[i,r]))
                                    file7.write('%.3f %.2f %.4f \n' %(phioarr[i], reqarr[r],  depths_ch[i,r]))

file1.close()
file2.close() 
file3.close()
file4.close()
file5.close()
file6.close()
file7.close()

In [ ]:
df = pd.DataFrame(agg_cs_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_minorax_ellipsoid_flat_ch.dat', sep='\t')

df = pd.DataFrame(agg_as_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipsoid_flat_ch.dat', sep='\t')

df = pd.DataFrame(agg_as_mean, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipsoid_flat_mean.dat', sep='\t')

df = pd.DataFrame(dds_mode, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_dd_flat_mode.dat', sep='\t')

df = pd.DataFrame(major_ax_zs_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipse_z_flat_ch.dat', sep='\t')

df = pd.DataFrame(major_ax_zs_mean, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipse_z_flat_mean.dat', sep='\t')

df = pd.DataFrame(depths_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_depth_flat_ch.dat', sep='\t')


In [ ]:
df = pd.DataFrame(agg_cs_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_minorax_ellipsoid_rand_ch.dat', sep='\t')

df = pd.DataFrame(agg_as_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipsoid_rand_ch.dat', sep='\t')

df = pd.DataFrame(agg_as_mean, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipsoid_rand_mean.dat', sep='\t')

df = pd.DataFrame(dds_mode, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_dd_rand_mode.dat', sep='\t')

df = pd.DataFrame(major_ax_zs_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipse_z_rand_ch.dat', sep='\t')

df = pd.DataFrame(major_ax_zs_mean, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_majorax_ellipse_z_rand_mean.dat', sep='\t')

df = pd.DataFrame(depths_ch, index=phioarr, columns=reqarr)
df.to_csv('../lookup_tables/ice_ice/oldformat_depth_rand_ch.dat', sep='\t')
